In this file, we will
- Run a hyperparameter tuning while training a model
- Log every hyperparameter and metrics in the MLflow UI
- Compare the results of the varioise runs in the MLflow UI
- Choose the best run and register it a model

In [21]:
# import relevant libraries

import mlflow
import mlflow.sklearn
from mlflow.models import infer_signature

import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import root_mean_squared_error

In [22]:
# loading the dataset
dataset = datasets.fetch_california_housing()

# dataset

In [23]:
# preparing the data
data = pd.DataFrame(dataset.data, columns=dataset.feature_names)
data['Price'] = dataset.target

data.head(n=10)

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,Price
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422
5,4.0368,52.0,4.761658,1.103627,413.0,2.139896,37.85,-122.25,2.697
6,3.6591,52.0,4.931907,0.951362,1094.0,2.128405,37.84,-122.25,2.992
7,3.1200,52.0,4.797527,1.061824,1157.0,1.788253,37.84,-122.25,2.414
8,2.0804,42.0,4.294118,1.117647,1206.0,2.026891,37.84,-122.26,2.267
9,3.6912,52.0,4.970588,0.990196,1551.0,2.172269,37.84,-122.25,2.611


In [24]:
# splitting the data into training and testing sets
X = data.drop(columns=['Price'])
y = data['Price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# defining the model
model = RandomForestRegressor(random_state=42)

# defining the hyperparameters for grid search
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10]
}

# setting up the grid search
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='neg_root_mean_squared_error', verbose=2, n_jobs=-1)

# fitting the model
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 27 candidates, totalling 135 fits


GridSearchCV(cv=5, estimator=RandomForestRegressor(random_state=42), n_jobs=-1,
             param_grid={'max_depth': [None, 10, 20],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [50, 100, 200]},
             scoring='neg_root_mean_squared_error', verbose=2)

In [31]:
# making predictions
best_model = grid_search.best_estimator_
y_pred = grid_search.predict(X_test)
best_y_pred = best_model.predict(X_test)

# evaluating the model
rmse = root_mean_squared_error(y_test, y_pred)
best_rmse = root_mean_squared_error(y_test, best_y_pred)
print(f"The Root Mean Square Error: {rmse}")
print(f"Best Model Root Mean Square Error: {best_rmse}")

The Root Mean Square Error: 0.5045727952039526
Best Model Root Mean Square Error: 0.5045727952039526


In [30]:
grid_search.best_params_

{'max_depth': 20, 'min_samples_split': 2, 'n_estimators': 200}

In [27]:
# creating the signature
signature = infer_signature(X_test, y_pred)

In [33]:
# creating MLflow experiment

from urllib.parse import urlparse
import mlflow.sklearn
import mlflow.sklearn

mlflow.set_experiment("California House Model")

with mlflow.start_run(run_name="random_forest_model"):
    mlflow.log_param("Model-Type", "Random Forest Regressor")

    # logging params
    best_model = grid_search.best_estimator_
    mlflow.log_param("Best N-estimator", grid_search.best_params_['n_estimators'])
    mlflow.log_param("Min Sample Split", grid_search.best_params_['min_samples_split'])
    mlflow.log_param("Max Depth", grid_search.best_params_['max_depth'])

    # logging metric
    mlflow.log_metric("RMSE", rmse)

    # logging the model
    mlflow.set_tracking_uri("http://127.0.0.1:5000")
    mlflow.set_registry_uri("http://127.0.0.1:5000")
    tracking_url_type = urlparse(mlflow.get_tracking_uri()).scheme
    if tracking_url_type != 'file':
        mlflow.sklearn.log_model(
            sk_model=best_model,
            artifact_path="model",
            registered_model_name="Best Random Forest Model"
        )
    else:
        mlflow.sklearn.log_model(sk_model=best_model, artifact_path="model", signature=signature)

print("Model Logging Successful")

2025/06/21 14:33:37 INFO mlflow.tracking.fluent: Experiment with name 'California House Model' does not exist. Creating a new experiment.
2025/06/21 14:33:45 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'Best Random Forest Model'.
2025/06/21 14:33:49 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: Best Random Forest Model, version 1


🏃 View run random_forest_model at: http://127.0.0.1:5000/#/experiments/2/runs/146df941470d4db98462964b8d8ed4c9
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/2
Model Logging Successful


Created version '1' of model 'Best Random Forest Model'.
